In [3]:
import json
import xlsxwriter
import pandas as pd
import os

In [9]:
lang_pair = "eng_hin"
filetype = "c4" # parallel_sample | gold
target_folder = f"alternative_data/c4/source_excel_{filetype}"

In [5]:
os.makedirs(target_folder, exist_ok=True)

In [6]:
with open(f"alternative_data/c4_augmented.json", "r") as f:
    sentences = json.load(f)

In [7]:
real_sentences = []
temp_ids = set()
for sent in sentences:
    if sent["sentence_id"] not in temp_ids:
        temp_ids.add(sent["sentence_id"])
        real_sentences.append({"sentence":sent["masked_sentence"].replace("[MASK]", sent["word"]),
                              "sentence_id": sent["sentence_id"],
                              "word_id": -1,
                              "probability": 0})

for sent in sentences:
    ct = 0
    for rep in sent["replacement"]:
        if rep["word"].strip().lower()!=sent["word"].lower():
            ct+=1
            real_sentences.append({"sentence":sent["masked_sentence"].replace("[MASK]", rep["word"].strip()),
                                  "sentence_id": sent["sentence_id"],
                                  "word_id": sent["word_id"],
                                  "probability": rep["prob"]})

df = pd.DataFrame(real_sentences).sort_values(["sentence_id", "word_id"]).reset_index(drop=True)
df

,sentence,sentence_id,word_id,probability
0,"October 30, 2017  Mancari offers stories of w...",0,-1,0.000
1,"September 30, 2017  Mancari offers stories of...",0,0,0.136
2,"June 30, 2017  Mancari offers stories of wand...",0,0,0.120
3,"April 30, 2017  Mancari offers stories of wan...",0,0,0.105
4,"March 30, 2017  Mancari offers stories of wan...",0,0,0.089
...,...,...,...,...
4118801,"Hosted by Sister Bearonce Knows, Bikes on Broa...",49999,24,0.985
4118802,"Hosted by Sister Bearonce Knows, Bikes on Broa...",49999,24,0.010
4118803,"Hosted by Sister Bearonce Knows, Bikes on Broa...",49999,24,0.003
4118804,"Hosted by Sister Bearonce Knows, Bikes on Broa...",49999,24,0.000


In [10]:
selected_sentences = sorted(real_sentences, key=lambda d: (d['sentence_id'], d["word_id"])) 

In [11]:
for s in selected_sentences:
    if "\n" in s["sentence"]:
        raise Exception

In [14]:
cur_len = 0
current_sentences = []
current_idx = 0
threshold = 10000000
for s in selected_sentences:
    sentence = s["sentence"]
    if cur_len+len(sentence)>threshold:
        print(cur_len)
        workbook = xlsxwriter.Workbook(f"{target_folder}/{current_idx}.xlsx")
        worksheet = workbook.add_worksheet()
        print(len(current_sentences))
        for i, el in enumerate(current_sentences):
            worksheet.write(f'A{i+1}', el)

        workbook.close()

        current_sentences = [sentence]
        current_idx += 1
        cur_len = len(sentence)
    else:
        current_sentences.append(sentence)
        cur_len += len(sentence)
workbook = xlsxwriter.Workbook(f"{target_folder}/{current_idx}.xlsx")
worksheet = workbook.add_worksheet()
print(len(current_sentences))
for i, el in enumerate(current_sentences):
    worksheet.write(f'A{i+1}', el)

workbook.close()


9999886
78543
9999984
80212
9999972
78870
9999971
78262
9999978
78183
9999850
79543
9999928
79967
9999828
79656
9999998
78527
9999911
78200
9999991
79156
9999999
77995
9999893
79262
9999873
79957
9999933
78711
9999962
78477
9999986
78771
9999935
78643
9999847
78753
9999925
79050
9999849
78644
9999941
78238
9999871
79518
9999925
79301
9999938
79044
9999894
78810
9999949
79443
9999966
79174
9999973
79407
9999989
78738
9999991
79110
9999946
78311
9999928
78718
9999939
79137
9999960
79500
9999963
78587
9999845
78625
9999942
78690
9999995
78992
9999876
78771
9999998
77681
9999907
78623
9999950
79521
9999958
78783
9999848
79873
9999961
80052
9999845
79202
9999977
78393
9999949
81614
9999949
78697
9999975
79157
9999943
77992
11722
